In [1]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

# keras imports
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.applications.xception import Xception, preprocess_input
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json

# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
#import cv2
import h5py
import os
import json
import pickle
import datetime
import time

Using TensorFlow backend.


In [2]:
import tensorflow as tf
tf.__version__

'1.2.1'

In [3]:
!jupyter kernelspec list
import sys
sys.executable

Available kernels:
  python3    C:\Users\hj5446\AppData\Local\Continuum\Anaconda3\envs\frac_detect\share\jupyter\kernels\python3


'C:\\Users\\hj5446\\AppData\\Local\\Continuum\\Anaconda3\\envs\\frac_detect\\python.exe'

In [4]:
# filter warnings
# load the user configs
with open('conf/conf_xception.json') as f:    
    config = json.load(f)
# config variables
model_name = config["model"]
weights = config["weights"]
include_top = config["include_top"]
train_path = config["train_path"]
features_path = config["features_path"]
labels_path = config["labels_path"]
test_size = config["test_size"]
results = config["results"]
model_path = config["model_path"]
#weights_path = config["weights_path"]
print('model_name {}. features_path {}'.format(model_name, features_path))

model_name xception. features_path output/flowers_17/xception/features.h5


In [6]:
# start time
print("[STATUS] start time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))
start = time.time()

# create the pretrained models
# check for pretrained weight usage or not
# check for top layers to be included or not

if model_name == "vgg16":
    base_model = VGG16(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('fc1').output)
    image_size = (224, 224)
elif model_name == "vgg19":
    base_model = VGG19(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('fc1').output)
    image_size = (224, 224)
elif model_name == "resnet50":
    base_model = ResNet50(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('flatten').output)
    image_size = (224, 224)
elif model_name == "inceptionv3":
    base_model = InceptionV3(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('avg_pool').output) # this is correct
    #model = Model(input=base_model.input, output=base_model.get_layer('Dense').output)
    image_size = (299, 299)
elif model_name == "xception":
    base_model = Xception(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('avg_pool').output)
    image_size = (299, 299)
else:
    base_model = None

print("[INFO] successfully loaded base model and model...")

[STATUS] start time - 2017-11-16 12:53
[INFO] successfully loaded base model and model...


C:\Users\hj5446\AppData\Local\Continuum\Anaconda3\envs\frac_detect\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("av...)`


In [7]:
# path to training dataset
train_files = os.listdir(train_path)

# encode the labels
#print("[INFO] encoding labels...")
#le = LabelEncoder()
#le.fit([tl for tl in train_labels])

# variables to hold features and labels
features = [] # store feature data, x
labels   = [] # store label data, y

label = 0
# loop over all the labels in the folder
for i, file in enumerate(train_files):
    img_path = train_path + "/" + file
    if i %10 == 0:
        print(' {}'.format(i), end=',')
    if (i) % 80 == 0 :
        label = label+1
        print('\n file {}, label {}'.format(file, label))    
    img = image.load_img(img_path, target_size=image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    flat = feature.flatten()
    features.append(flat)
    labels.append(label)
print("[INFO] processed - {}".format(i))
print( "[INFO] completed label - {}".format(label))
print(len(labels))

i0,file image_0001.jpg, label 1
i10,i20,i30,i40,i50,i60,i70,i80,file image_0081.jpg, label 2
i90,i100,i110,i120,i130,i140,i150,i160,file image_0161.jpg, label 3
i170,i180,i190,i200,i210,i220,i230,i240,file image_0241.jpg, label 4
i250,i260,i270,i280,i290,i300,i310,i320,file image_0321.jpg, label 5
i330,i340,i350,i360,i370,i380,i390,i400,file image_0401.jpg, label 6
i410,i420,i430,i440,i450,i460,i470,i480,file image_0481.jpg, label 7
i490,i500,i510,i520,i530,i540,i550,i560,file image_0561.jpg, label 8
i570,i580,i590,i600,i610,i620,i630,i640,file image_0641.jpg, label 9
i650,i660,i670,i680,i690,i700,i710,i720,file image_0721.jpg, label 10
i730,i740,i750,i760,i770,i780,i790,i800,file image_0801.jpg, label 11
i810,i820,i830,i840,i850,i860,i870,i880,file image_0881.jpg, label 12
i890,i900,i910,i920,i930,i940,i950,i960,file image_0961.jpg, label 13
i970,i980,i990,i1000,i1010,i1020,i1030,i1040,file image_1041.jpg, label 14
i1050,i1060,i1070,i1080,i1090,i1100,i1110,i1120,file image_1121.jpg, l

In [8]:
# encode the labels using LabelEncoder
targetNames = np.unique(labels)
#le = LabelEncoder()
#le_labels = le.fit_transform(labels)
le_labels = np.array(labels)
# get the shape of training labels
print("[STATUS] training labels: {}".format(le_labels))
print("[STATUS] training labels shape: {}".format(le_labels.shape))

# save features and labels
h5f_data = h5py.File(features_path, 'w')
h5f_data.create_dataset('dataset_1', data=np.array(features))

h5f_label = h5py.File(labels_path, 'w')
h5f_label.create_dataset('dataset_1', data=np.array(le_labels))

h5f_data.close()
h5f_label.close()

# save model and weights
model_json = model.to_json()
with open(model_path + str(test_size) + ".json", "w") as json_file:
    json_file.write(model_json)

# save weights
model.save_weights(model_path + str(test_size) + ".h5")
print("[STATUS] saved model and weights to disk..")

print("[STATUS] features and labels saved..")

# end time
end = time.time()
print("[STATUS] end time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))

[STATUS] training labels: [ 1  1  1 ..., 17 17 17]
[STATUS] training labels shape: (1360,)
[STATUS] saved model and weights to disk..
[STATUS] features and labels saved..
[STATUS] end time - 2017-11-16 13:07
